In [3]:
import argparse
from dataclasses import dataclass
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from transformers import pipeline

In [4]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Générer une histoire de jeux vidéo en se basant sur cette phrase : "## {question} ##" et selon le contexte suivant :

{context}

---

Générer une histoire de jeux vidéo en se basant sur cette phrase : "## {question} ##" et selon le contexte donné ci-dessus
"""

In [6]:
# Function to handle the main logic
def main(query_text):
    # Prepare the DB.
    embedding_function = HuggingFaceEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=1)
    #if len(results) == 0 or results[0][1] < 0.7:
    #    print(f"Unable to find matching results.")
    #    return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    #corrected_context = correct_encoding(context_text)
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print("Prompt:", prompt)

    generator = pipeline('text-generation', model='DLProjectLSI/Phi-3-mini-4k_16bit')
    response = generator(prompt, max_length=1024, num_return_sequences=1,truncation=True)
    response_text = response[0]['generated_text']

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)